In [20]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Read Multiple Files').getOrCreate()

In [32]:
from pyspark.sql.functions import input_file_name, regexp_extract, initcap, col
from concurrent.futures import ThreadPoolExecutor

def read_write_csv_file(country):
    df = spark.read.csv(f'data/{country}.csv', header=True, inferSchema=True)
    df = df.withColumn("country", initcap(regexp_extract(input_file_name(), r"([^/]+)\.csv", 1)))

    string_fields = [c for c, t in df.dtypes if t == 'string']

    for c in string_fields:
        df = df.withColumn(c, initcap(col(c)))
    
    df.write.mode('overwrite').csv(f'data/{country}-output.csv', header=True)
    
countries = ['india', 'usa']

with ThreadPoolExecutor(max_workers=1) as executor:
    executor.map(read_write_csv_file, countries)

26/01/17 03:02:32 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1051206 ms exceeds timeout 120000 ms
26/01/17 03:02:32 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/17 03:20:22 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:674)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1363)
	at 

In [ ]:
# Load in output file

In [17]:
spark.stop()